# INSERT screenshots and/or terminal output to demonstrate the process of activating the hadoop cluster
# then adding the files to HDFS

In [58]:
import pyspark

import findspark

from pyspark.sql import SparkSession

import pandas as pd
import os

# os.environ['SPARK_HOME']

# os.environ['SPARK_HOME'] = '/opt/spark/'

print(os.environ['PATH'])

findspark.init('/opt/spark/')

spark = SparkSession.builder.appName('SparkExample').config('spark.driver.python', '/snap/jupyter/6/bin/python3').getOrCreate()

import sys

print(sys.executable)

print(sys.prefix)

/home/hduser/env/myenv/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin:/usr/local/hadoop/bin:/bin:/usr/local/hadoop/sbin:/usr/local/hadoop/bin:/bin:/usr/local/hadoop/sbin:/opt/spark/bin:/usr/local/hadoop/bin:/bin:/usr/local/hadoop/sbin:/opt/spark/bin
/home/hduser/env/myenv/bin/python
/home/hduser/env/myenv


In [17]:
dfs = spark.read.csv("hdfs://localhost:9000/transactions/*.csv", header=True, inferSchema=True)

In [13]:
dfs.show()

+---------+---------+------------+---------+--------+-----+----------+------------+------------+------------+------------+--------------+-----+-----+---------------+-----------+--------------------+-----------+--------------------+----------+------------+------------+
|      _c0|      _c1|         _c2|      _c3|     _c4|  _c5|       _c6|         _c7|         _c8|         _c9|        _c10|          _c11| _c12| _c13|           _c14|       _c15|                _c16|       _c17|                _c18|      _c19|        _c20|        _c21|
+---------+---------+------------+---------+--------+-----+----------+------------+------------+------------+------------+--------------+-----+-----+---------------+-----------+--------------------+-----------+--------------------+----------+------------+------------+
|SHOP_WEEK|SHOP_DATE|SHOP_WEEKDAY|SHOP_HOUR|QUANTITY|SPEND| PROD_CODE|PROD_CODE_10|PROD_CODE_20|PROD_CODE_30|PROD_CODE_40|     CUST_CODE|seg_1|seg_2|      BASKET_ID|BASKET_SIZE|BASKET_PRICE_SEN

In [18]:
dfs.printSchema()

root
 |-- SHOP_WEEK: string (nullable = true)
 |-- SHOP_DATE: string (nullable = true)
 |-- SHOP_WEEKDAY: string (nullable = true)
 |-- SHOP_HOUR: string (nullable = true)
 |-- QUANTITY: string (nullable = true)
 |-- SPEND: string (nullable = true)
 |-- PROD_CODE: string (nullable = true)
 |-- PROD_CODE_10: string (nullable = true)
 |-- PROD_CODE_20: string (nullable = true)
 |-- PROD_CODE_30: string (nullable = true)
 |-- PROD_CODE_40: string (nullable = true)
 |-- CUST_CODE: string (nullable = true)
 |-- seg_1: string (nullable = true)
 |-- seg_2: string (nullable = true)
 |-- BASKET_ID: string (nullable = true)
 |-- BASKET_SIZE: string (nullable = true)
 |-- BASKET_PRICE_SENSITIVITY: string (nullable = true)
 |-- BASKET_TYPE: string (nullable = true)
 |-- BASKET_DOMINANT_MISSION: string (nullable = true)
 |-- STORE_CODE: string (nullable = true)
 |-- STORE_FORMAT: string (nullable = true)
 |-- STORE_REGION: string (nullable = true)



### Perform groupby operations to aggregate data by date for:
##### total spend by store code
##### total spend by store format
##### total spend by prod code / prod_code_10 / prod_code_20 etc - depending on the highest level of granularity during EDA


In [21]:
dfs.createOrReplaceTempView("STORE_DATA")

In [36]:
store_df = spark.sql("SELECT SHOP_DATE, STORE_CODE, SUM(QUANTITY) AS TOTAL_QTY, ROUND(SUM(SPEND), 2) AS TOTAL_SPEND FROM STORE_DATA GROUP BY SHOP_DATE, STORE_CODE")

In [37]:
store_df.show()

23/09/12 06:56:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 06:56:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


+---------+----------+---------+-----------+
|SHOP_DATE|STORE_CODE|TOTAL_QTY|TOTAL_SPEND|
+---------+----------+---------+-----------+
| 20061223|STORE00003|    215.0|     316.65|
| 20061219|STORE00113|    235.0|     340.55|
| 20061221|STORE00120|     21.0|      70.53|
| 20061224|STORE00227|     73.0|     164.17|
| 20061224|STORE00432|     86.0|     118.75|
| 20061219|STORE00494|     13.0|      10.78|
| 20061221|STORE00633|    144.0|     228.09|
| 20061221|STORE00639|    155.0|      178.8|
| 20061222|STORE01091|      9.0|      12.73|
| 20061220|STORE01215|    125.0|     202.03|
| 20061218|STORE01289|     36.0|      37.31|
| 20061222|STORE01330|     24.0|      22.07|
| 20061222|STORE01603|     21.0|       27.4|
| 20061224|STORE01651|    113.0|     137.57|
| 20061222|STORE01672|     41.0|      53.55|
| 20061221|STORE01707|    264.0|     350.76|
| 20061223|STORE01793|     23.0|      18.85|
| 20061223|STORE02524|    147.0|     200.77|
| 20061219|STORE02575|      8.0|      28.24|
| 20061220

In [38]:
store_format_df = spark.sql("SELECT SHOP_DATE, STORE_CODE, STORE_FORMAT, SUM(QUANTITY) AS TOTAL_QTY, ROUND(SUM(SPEND), 2) AS TOTAL_SPEND FROM STORE_DATA GROUP BY SHOP_DATE, STORE_CODE, STORE_FORMAT")

In [39]:
store_format_df.show()

23/09/12 06:58:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, STORE_CODE, STORE_FORMAT
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE, STORE_FORMAT
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 06:58:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE, STORE_FORMAT
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


+---------+----------+------------+---------+-----------+
|SHOP_DATE|STORE_CODE|STORE_FORMAT|TOTAL_QTY|TOTAL_SPEND|
+---------+----------+------------+---------+-----------+
| 20061218|STORE00107|          LS|     98.0|     153.96|
| 20061218|STORE00120|          MS|      8.0|      22.61|
| 20061218|STORE00422|          MS|     78.0|     147.67|
| 20061221|STORE00425|          MS|     63.0|     118.09|
| 20061221|STORE00460|          LS|    217.0|     317.46|
| 20061221|STORE00807|          LS|     66.0|      79.44|
| 20061222|STORE00840|          LS|    325.0|     487.22|
| 20061221|STORE00877|          LS|    290.0|     487.31|
| 20061223|STORE00895|          SS|      6.0|       8.73|
| 20061222|STORE00966|          MS|    145.0|     194.21|
| 20061218|STORE01035|          LS|     16.0|      16.61|
| 20061219|STORE01160|          LS|    108.0|     136.29|
| 20061224|STORE01298|          SS|     26.0|      26.28|
| 20061221|STORE01316|          LS|    245.0|     429.21|
| 20061221|STO

In [42]:
prod_10_df = spark.sql("SELECT SHOP_DATE, STORE_CODE, PROD_CODE_10, SUM(QUANTITY) AS TOTAL_QTY, ROUND(SUM(SPEND), 2) AS TOTAL_SPEND FROM STORE_DATA GROUP BY SHOP_DATE, STORE_CODE, PROD_CODE_10")

In [43]:
prod_10_df.show()

23/09/12 07:03:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_10, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_10, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:03:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_10, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


+---------+----------+------------+---------+-----------+
|SHOP_DATE|STORE_CODE|PROD_CODE_10|TOTAL_QTY|TOTAL_SPEND|
+---------+----------+------------+---------+-----------+
| 20060410|STORE00001|     CL00012|      1.0|       0.01|
| 20060410|STORE00001|     CL00020|      5.0|        2.2|
| 20060410|STORE00001|     CL00201|      1.0|       0.81|
| 20060410|STORE00002|     CL00001|      2.0|       1.72|
| 20060410|STORE00003|     CL00023|      1.0|       1.93|
| 20060410|STORE00003|     CL00065|      1.0|       1.16|
| 20060410|STORE00003|     CL00074|      2.0|       5.22|
| 20060410|STORE00003|     CL00128|      1.0|       1.02|
| 20060410|STORE00003|     CL00135|      1.0|       0.16|
| 20060410|STORE00003|     CL00157|      2.0|       1.58|
| 20060410|STORE00003|     CL00161|      3.0|        3.3|
| 20060410|STORE00004|     CL00017|      1.0|       3.35|
| 20060410|STORE00004|     CL00043|      1.0|       0.97|
| 20060410|STORE00004|     CL00063|      6.0|       6.64|
| 20060410|STO

In [44]:
prod_20_df = spark.sql("SELECT SHOP_DATE, STORE_CODE, PROD_CODE_20, SUM(QUANTITY) AS TOTAL_QTY, ROUND(SUM(SPEND), 2) AS TOTAL_SPEND FROM STORE_DATA GROUP BY SHOP_DATE, STORE_CODE, PROD_CODE_20")

In [45]:
prod_20_df.show()

23/09/12 07:05:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_20, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_20, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:05:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_20, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


+---------+----------+------------+---------+-----------+
|SHOP_DATE|STORE_CODE|PROD_CODE_20|TOTAL_QTY|TOTAL_SPEND|
+---------+----------+------------+---------+-----------+
| 20060410|STORE00001|    DEP00003|      1.0|       0.01|
| 20060410|STORE00001|    DEP00005|      5.0|        2.2|
| 20060410|STORE00001|    DEP00025|      3.0|       2.61|
| 20060410|STORE00001|    DEP00051|      3.0|       3.97|
| 20060410|STORE00001|    DEP00067|      1.0|       0.81|
| 20060410|STORE00001|    DEP00070|      1.0|       0.67|
| 20060410|STORE00002|    DEP00008|      2.0|       2.21|
| 20060410|STORE00002|    DEP00011|      1.0|        0.4|
| 20060410|STORE00003|    DEP00008|     13.0|      17.66|
| 20060410|STORE00003|    DEP00016|      1.0|       5.14|
| 20060410|STORE00003|    DEP00021|     11.0|      16.59|
| 20060410|STORE00003|    DEP00025|      4.0|       4.83|
| 20060410|STORE00003|    DEP00069|      1.0|       1.87|
| 20060410|STORE00003|    DEP00073|      3.0|       1.87|
| 20060410|STO

In [46]:
prod_30_df = spark.sql("SELECT SHOP_DATE, STORE_CODE, PROD_CODE_30, SUM(QUANTITY) AS TOTAL_QTY, ROUND(SUM(SPEND), 2) AS TOTAL_SPEND FROM STORE_DATA GROUP BY SHOP_DATE, STORE_CODE, PROD_CODE_30")

In [47]:
prod_30_df.show()

23/09/12 07:07:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_30, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_30, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:07:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_30, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


+---------+----------+------------+---------+-----------+
|SHOP_DATE|STORE_CODE|PROD_CODE_30|TOTAL_QTY|TOTAL_SPEND|
+---------+----------+------------+---------+-----------+
| 20060410|STORE00001|      G00002|      1.0|       0.01|
| 20060410|STORE00001|      G00003|      5.0|        2.2|
| 20060410|STORE00001|      G00006|      1.0|       2.58|
| 20060410|STORE00001|      G00010|      3.0|       1.02|
| 20060410|STORE00001|      G00015|      4.0|       5.26|
| 20060410|STORE00002|      G00003|      1.0|       2.05|
| 20060410|STORE00002|      G00011|      1.0|       0.41|
| 20060410|STORE00002|      G00016|      3.0|       3.64|
| 20060410|STORE00002|      G00021|      1.0|        1.4|
| 20060410|STORE00002|      G00023|      1.0|       0.31|
| 20060410|STORE00003|      G00002|      2.0|       2.11|
| 20060410|STORE00003|      G00003|      2.0|        2.2|
| 20060410|STORE00003|      G00004|     20.0|      23.56|
| 20060410|STORE00003|      G00007|     47.0|      67.46|
| 20060410|STO

In [48]:
prod_40_df = spark.sql("SELECT SHOP_DATE, STORE_CODE, PROD_CODE_40, SUM(QUANTITY) AS TOTAL_QTY, ROUND(SUM(SPEND), 2) AS TOTAL_SPEND FROM STORE_DATA GROUP BY SHOP_DATE, STORE_CODE, PROD_CODE_40")

In [49]:
prod_40_df.show()

23/09/12 07:09:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_40, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_40, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:09:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_40, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


+---------+----------+------------+---------+-----------+
|SHOP_DATE|STORE_CODE|PROD_CODE_40|TOTAL_QTY|TOTAL_SPEND|
+---------+----------+------------+---------+-----------+
| 20061219|STORE00027|      D00002|     40.0|      58.04|
| 20061218|STORE00030|      D00001|      7.0|       8.51|
| 20061221|STORE00065|      D00004|      4.0|       7.92|
| 20061218|STORE00327|      D00002|     39.0|      59.88|
| 20061222|STORE00327|      D00002|     52.0|      77.29|
| 20061219|STORE00337|      D00001|      4.0|       4.88|
| 20061223|STORE00352|      D00003|      7.0|       5.06|
| 20061218|STORE00377|      D00002|     84.0|     114.26|
| 20061224|STORE00377|      D00004|      2.0|       4.18|
| 20061219|STORE00468|      D00001|      2.0|       2.25|
| 20061218|STORE00500|      D00009|      1.0|       1.35|
| 20061223|STORE00508|      D00001|      5.0|       2.16|
| 20061224|STORE00560|      D00003|     16.0|       11.8|
| 20061219|STORE00574|      D00001|     23.0|      16.83|
| 20061223|STO

In [50]:
prod_40_df.printSchema()

root
 |-- SHOP_DATE: string (nullable = true)
 |-- STORE_CODE: string (nullable = true)
 |-- PROD_CODE_40: string (nullable = true)
 |-- TOTAL_QTY: double (nullable = true)
 |-- TOTAL_SPEND: double (nullable = true)



## Find the estimated size of each of the dataframes

In [54]:
def estimate_df_memory_usage(df)-> None:
    sample = df.sample(fraction=0.01)
    pdf = sample.toPandas()
    print(pdf.info())

In [60]:
estimate_df_memory_usage(store_df)

23/09/12 07:34:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:34:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6057 entries, 0 to 6056
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SHOP_DATE    6057 non-null   object 
 1   STORE_CODE   6056 non-null   object 
 2   TOTAL_QTY    6056 non-null   float64
 3   TOTAL_SPEND  6056 non-null   float64
dtypes: float64(2), object(2)
memory usage: 189.4+ KB
None


### Estimated df size = 189.4 kb * 100 = 18.9MB

In [61]:
estimate_df_memory_usage(store_format_df)

23/09/12 07:36:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, STORE_CODE, STORE_FORMAT
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE, STORE_FORMAT
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:36:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, STORE_CODE, STORE_FORMAT
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6005 entries, 0 to 6004
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SHOP_DATE     6005 non-null   object 
 1   STORE_CODE    6004 non-null   object 
 2   STORE_FORMAT  6004 non-null   object 
 3   TOTAL_QTY     6004 non-null   float64
 4   TOTAL_SPEND   6004 non-null   float64
dtypes: float64(2), object(3)
memory usage: 234.7+ KB
None


### Estimated df size = 235kb * 100 = 23.5MB

In [62]:
estimate_df_memory_usage(prod_10_df)

23/09/12 07:39:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_10, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_10, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:39:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_10, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212111 entries, 0 to 212110
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   SHOP_DATE     212111 non-null  object 
 1   STORE_CODE    212109 non-null  object 
 2   PROD_CODE_10  212109 non-null  object 
 3   TOTAL_QTY     212109 non-null  float64
 4   TOTAL_SPEND   212109 non-null  float64
dtypes: float64(2), object(3)
memory usage: 8.1+ MB
None


### Estimated df size = 8.1MB * 100 = 810MB

In [63]:
estimate_df_memory_usage(prod_20_df)

23/09/12 07:42:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_20, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_20, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:42:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_20, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136880 entries, 0 to 136879
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   SHOP_DATE     136880 non-null  object 
 1   STORE_CODE    136879 non-null  object 
 2   PROD_CODE_20  136879 non-null  object 
 3   TOTAL_QTY     136879 non-null  float64
 4   TOTAL_SPEND   136879 non-null  float64
dtypes: float64(2), object(3)
memory usage: 5.2+ MB
None


### Estimated df size = 5.2MB * 100 = 520MB

In [64]:
estimate_df_memory_usage(prod_30_df)

23/09/12 07:44:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_30, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_30, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:44:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_30, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77700 entries, 0 to 77699
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SHOP_DATE     77700 non-null  object 
 1   STORE_CODE    77698 non-null  object 
 2   PROD_CODE_30  77698 non-null  object 
 3   TOTAL_QTY     77698 non-null  float64
 4   TOTAL_SPEND   77698 non-null  float64
dtypes: float64(2), object(3)
memory usage: 3.0+ MB
None


### Estimated df size = 3MB * 100 = 300MB

In [59]:
estimate_df_memory_usage(prod_40_df)

23/09/12 07:31:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SHOP_DATE, QUANTITY, SFNND, PROD_CODE_40, STORE_CODE
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_40, STORE_CODE
Expected: SPEND but found: SFNND
CSV file: hdfs://localhost:9000/transactions/transactions_200607.csv
23/09/12 07:31:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: date_from, , , , 
 Schema: SHOP_DATE, QUANTITY, SPEND, PROD_CODE_40, STORE_CODE
Expected: SHOP_DATE but found: date_from
CSV file: hdfs://localhost:9000/transactions/time.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28393 entries, 0 to 28392
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SHOP_DATE     28393 non-null  object 
 1   STORE_CODE    28393 non-null  object 
 2   PROD_CODE_40  28393 non-null  object 
 3   TOTAL_QTY     28393 non-null  float64
 4   TOTAL_SPEND   28393 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.1+ MB
None


### Estimated df size = 1.1MB * 100 = 110MB

### Explore EDA capabilities of pyspark - is it better to use pandas on the data directly?
### How useful is spark for feature engineering vs python?
### Is it better to build NN in spark (does spark support keras/tensorflow) or in python?